In [8]:
#Advait Deochakke
#Lab 1 - Web Mining 
#Inverted Index

This code is for Lab 1 of CSE3024 - Web Mining

by Advait Deochakke - 20BCE1143

Inverted Indices and Incidence Matrices

In [9]:
import os
import string

We import OS and String libraries to go through the files,


and work with the documents under each folder

Below are functions written to generalize getting the inverted indices for all documents in a folder


Our function not only gets the occurence, but also the line number in that document

In [10]:

def build_inverted_index(folder_path):
    # Initialize an empty dictionary to store the inverted index
    inverted_index = {}

    # Iterate over the files in the "q1" folder
    for filename in os.listdir(folder_path):
        # Open the file
        with open(os.path.join(folder_path, filename), 'r') as f:
            # Read the contents of the file into a list of lines
            lines = f.readlines()

        # Iterate over the lines in the file
        for line_number, line in enumerate(lines):
            # Split the line into a list of words
            words = line.split()

            # Iterate over the words in the line
            for word in words:
                # Normalize the word (lowercase and remove punctuation)
                normalized_word = word.lower().strip(string.punctuation)

                # If the word is not in the inverted index, add it and set its value to an empty dictionary
                if normalized_word not in inverted_index:
                    inverted_index[normalized_word] = {}

                # If the file is not in the dictionary for this word, add it and set its value to an empty list
                if filename not in inverted_index[normalized_word]:
                    inverted_index[normalized_word][filename] = []

                # Add the line number to the list of line numbers for this word and file
                inverted_index[normalized_word][filename].append(line_number)
    return inverted_index

def print_inverted_index(inverted_index):
    # Iterate over the items in the inverted index
    for word, line_numbers in inverted_index.items():
        # Print the word and the line numbers it appears on
        print(f"{word}: {line_numbers}")


Below we have the code for Incidence Matrices

We use the generated ivnerted index to get the incidence matrix

Then print it with justification

In [11]:

def build_incidence_matrix(inverted_index):
    # Create an empty list of lists to store the matrix
    matrix = []

    # Get the list of documents from the inverted index
    documents = []
    for line_numbers_by_document in inverted_index.values():
        documents += list(line_numbers_by_document.keys())
    documents = list(set(documents))

    # Add the file names as the first row of the matrix
    matrix.append([""] + list(documents))

    # Iterate over the items in the inverted index
    for word, line_numbers_by_document in inverted_index.items():
        # Initialize an empty list to store the row for this word
        row = [word]

        # Iterate over the documents
        for document in documents:
            # If the word appears in the document, add a 1 to the row
            if document in line_numbers_by_document:
                row.append(1)
            # If the word does not appear in the document, add a 0 to the row
            else:
                row.append(0)

        # Add the row to the matrix
        matrix.append(row)
    return matrix

def print_incidence_matrix(matrix):
    # Determine the maximum length of the values in each column
    column_widths = [max(len(str(row[i])) for row in matrix) for i in range(len(matrix[0]))]

    # Print the matrix
    for row in matrix:
        print(" ".join(str(x).ljust(width) for x, width in zip(row, column_widths)))


Below we have consolidatory code, to wrap everything in a nice package

We also add the code for the boolean queries into the incidence matrix

In [12]:

def build_and_print_both(folder_path):
    inverted_index = build_inverted_index(folder_path=folder_path)
    matrix = build_incidence_matrix(inverted_index)
    print("inverted index for "+ folder_path)
    print_inverted_index(inverted_index)
    print("\nincidence matrix for " + folder_path)
    print_incidence_matrix(matrix)
    print("\n\n")
    return matrix

def boolean_query(matrix, words, operator):
    # Handle the "NOT" operator separately
    if operator == "NOT":
        # Initialize an empty list to store the results
        results = []

        # Iterate over the rows of the matrix
        for row in matrix:
            # If the row corresponds to one of the words in the query, get the list of documents that do not contain the word
            if row[0] in words:
                documents = [i for i, x in enumerate(row[1:]) if x == 0]
                # Union the results with the list of documents
                results = list(set(results).union(documents))

        # Return the results
        return results

    # Initialize an empty list to store the results
    results = []

    # Iterate over the rows of the matrix
    for row in matrix:
        # If the row corresponds to one of the words in the query
        if row[0] in words:
            # Get the list of documents that contain the word
            documents = [i for i, x in enumerate(row[1:]) if x == 1]

            # If this is the first row, set the results to the list of documents
            if not results:
                results = documents
            # If the operator is "AND", intersect the results with the list of documents
            elif operator == "AND":
                results = list(set(results).intersection(documents))
            # If the operator is "OR", union the results with the list of documents
            elif operator == "OR":
                results = list(set(results).union(documents))

    # Return the results
    return results

def print_part_three(q2matrix):    
    print("schizophrenia AND drug :", end=" ")
    print(boolean_query(q2matrix, ["schizophrenia", "drug"], "AND"))

    print("hope OR breakthrough :", end=" ")
    print(boolean_query(q2matrix, ["hope", "breakthrough"], "OR"))

    # Compute the result of the query 'word2 OR word3'
    or_result = boolean_query(q2matrix, ["drug", "approach"], "OR")

    # Compute the result of the query 'NOT (word2 OR word3)' by negating the result
    not_result = [i for i in range(len(q2matrix[0])) if i not in or_result]

    # Compute the final result by intersecting the result of the query 'word1 AND NOT (word2 OR word3)' with the result of the query 'NOT (word2 OR word3)'
    and_result = boolean_query(q2matrix, ["for"], "AND")
    final_result = list(set(and_result).intersection(not_result))

    # Print the final result
    print("for AND NOT (drug OR approach) :", end=" ")
    print(final_result)


Below we show the execution for the code, using the directories q1 and q2

In [13]:
#---------------------------------------------------------------------------------------
#part A
q1matrix = build_and_print_both('q1')

inverted index for q1
new: {'doc1.txt': [0], 'doc4.txt': [0]}
home: {'doc1.txt': [0], 'doc2.txt': [0], 'doc3.txt': [0], 'doc4.txt': [0]}
sales: {'doc1.txt': [0], 'doc2.txt': [0], 'doc3.txt': [0], 'doc4.txt': [0]}
top: {'doc1.txt': [0]}
forecasts: {'doc1.txt': [0]}
rise: {'doc2.txt': [0], 'doc4.txt': [0]}
in: {'doc2.txt': [0], 'doc3.txt': [0, 0]}
july: {'doc2.txt': [0], 'doc3.txt': [0], 'doc4.txt': [0]}
increase: {'doc3.txt': [0]}

incidence matrix for q1
          doc1.txt doc4.txt doc3.txt doc2.txt
new       1        1        0        0       
home      1        1        1        1       
sales     1        1        1        1       
top       1        0        0        0       
forecasts 1        0        0        0       
rise      0        1        0        1       
in        0        0        1        1       
july      0        1        1        1       
increase  0        0        1        0       





In [14]:

#---------------------------------------------------------------------------------------
#part B
q2matrix = build_and_print_both('q2')


#---------------------------------------------------------------------------------------


inverted index for q2
breakthrough: {'doc1.txt': [0]}
drug: {'doc1.txt': [0], 'doc2.txt': [0]}
for: {'doc1.txt': [0], 'doc3.txt': [0], 'doc4.txt': [0]}
schizophrenia: {'doc1.txt': [0], 'doc2.txt': [0], 'doc3.txt': [0], 'doc4.txt': [0]}
new: {'doc2.txt': [0], 'doc3.txt': [0], 'doc4.txt': [0]}
approach: {'doc3.txt': [0]}
treatment: {'doc3.txt': [0]}
of: {'doc3.txt': [0]}
hopes: {'doc4.txt': [0]}
patients: {'doc4.txt': [0]}

incidence matrix for q2
              doc1.txt doc4.txt doc3.txt doc2.txt
breakthrough  1        0        0        0       
drug          1        0        0        1       
for           1        1        1        0       
schizophrenia 1        1        1        1       
new           0        1        1        1       
approach      0        0        1        0       
treatment     0        0        1        0       
of            0        0        1        0       
hopes         0        1        0        0       
patients      0        1        0        0       


In [15]:
print_part_three(q2matrix)

schizophrenia AND drug : [0, 3]
hope OR breakthrough : [0]
for AND NOT (drug OR approach) : [1]
